In [1]:
# Import required libraries
import pydeck as pdk
import pandas as pd
import requests
from datetime import datetime, timedelta
import json
import ipywidgets as widgets
from IPython.display import display

print("✓ Libraries imported successfully")


✓ Libraries imported successfully


# 10m Amateur Radio Contacts - Interactive GreatCircle Visualization

This notebook visualizes 10m amateur radio contacts using pydeck's GreatCircleLayer.  
Data is queried from the DX Cluster API at api.jxqz.org:8080.

**Features:**
- Interactive checkboxes to filter FM and/or SSB contacts
- Real-time data from today's propagation
- Great circle paths showing radio propagation
- Interactive tooltips with contact details

## Fetch 10m Contacts from API

Query the DX Cluster API for today's 10m contacts and filter based on your selections.

**Frequency Ranges:**
- **FM:** 29.600 - 29.700 MHz (10m FM calling frequencies)
- **SSB:** 28.300 - 29.700 MHz (10m SSB portion)

In [3]:
# API endpoint
API_BASE = "http://api.jxqz.org:8080/api"

# Calculate today's date range
today = datetime.now().date()
since = f"{today}T00:00:00"

# Query parameters for 10m contacts
params = {
    'band': '10m',
    'since': since,
    'limit': 500
}

print(f"Querying API for 10m contacts since {since}...")
response = requests.get(f"{API_BASE}/spots", params=params, timeout=10)
data = response.json()

all_spots = data.get('spots', [])
print(f"Retrieved {len(all_spots)} total 10m spots")

# Filter based on checkboxes
filtered_spots = []

if fm_checkbox.value:
    # FM frequency range: 29.6-29.7 MHz (29600-29700 kHz)
    for spot in all_spots:
        try:
            freq = float(spot['frequency'])
            if 29600 <= freq <= 29700:
                filtered_spots.append(spot)
        except (ValueError, KeyError):
            continue
    print(f"Found {len(filtered_spots)} FM spots (29.6-29.7 MHz)")

if ssb_checkbox.value:
    # SSB frequency range: 28.3-29.7 MHz (28300-29700 kHz), excluding FM range
    ssb_count = 0
    for spot in all_spots:
        try:
            freq = float(spot['frequency'])
            # SSB range, but exclude FM frequencies if FM is also checked
            if 28300 <= freq <= 29700:
                if fm_checkbox.value and 29600 <= freq <= 29700:
                    continue  # Skip FM range if FM is checked
                if spot not in filtered_spots:  # Avoid duplicates
                    filtered_spots.append(spot)
                    ssb_count += 1
        except (ValueError, KeyError):
            continue
    print(f"Found {ssb_count} SSB spots (28.3-29.7 MHz)")

spots = filtered_spots

# If no spots with selected filters, show message
if len(spots) == 0:
    if not fm_checkbox.value and not ssb_checkbox.value:
        print("\n⚠ No filter selected. Please check FM and/or SSB above.")
    else:
        print("\n⚠ No spots found with selected filters. Showing all 10m contacts...")
        spots = all_spots

# Display first few spots
if spots:
    print(f"\nTotal spots to visualize: {len(spots)}")
    print("\nSample spots:")
    for i, spot in enumerate(spots[:5]):
        print(f"  {i+1}. {spot['dx_call']} @ {spot['frequency']} kHz spotted by {spot['spotter_call']} - {spot.get('mode', 'N/A')}")
else:
    print("\nNo spots found for today. The band may be quiet.")

Querying API for 10m contacts since 2025-12-06T00:00:00...
Retrieved 500 total 10m spots
Found 11 FM spots (29.6-29.7 MHz)
Found 354 SSB spots (28.3-29.7 MHz)

Total spots to visualize: 365

Sample spots:
  1. XE2ARP @ 29600.000 kHz spotted by N9GXA - None
  2. NZ9N @ 29600.000 kHz spotted by KF4WE - None
  3. N7GB/M @ 29600.000 kHz spotted by KF4WE - None
  4. WA1YKL @ 29600.000 kHz spotted by KF4WE - None
  5. N5PP @ 29600.000 kHz spotted by WA3LKT - None
Retrieved 500 total 10m spots
Found 11 FM spots (29.6-29.7 MHz)
Found 354 SSB spots (28.3-29.7 MHz)

Total spots to visualize: 365

Sample spots:
  1. XE2ARP @ 29600.000 kHz spotted by N9GXA - None
  2. NZ9N @ 29600.000 kHz spotted by KF4WE - None
  3. N7GB/M @ 29600.000 kHz spotted by KF4WE - None
  4. WA1YKL @ 29600.000 kHz spotted by KF4WE - None
  5. N5PP @ 29600.000 kHz spotted by WA3LKT - None


In [2]:
# Create interactive checkboxes for filtering
fm_checkbox = widgets.Checkbox(
    value=True,
    description='FM (29.6-29.7 MHz)',
    style={'description_width': 'initial'}
)

ssb_checkbox = widgets.Checkbox(
    value=True,
    description='SSB (28.3-29.7 MHz)',
    style={'description_width': 'initial'}
)

# Display checkboxes
display(widgets.VBox([
    widgets.HTML("<b>Select contact types to visualize:</b>"),
    fm_checkbox,
    ssb_checkbox
]))

print("✓ Filters ready. Run the next cell to fetch data with selected filters.")


✓ Filters ready. Run the next cell to fetch data with selected filters.


## Interactive Filters

Select which types of contacts to visualize:

## Callsign to Coordinates Lookup

We'll use QRZ.com's API or HamDB for callsign lookups. For this demo, we'll use a basic geographic estimate based on callsign prefixes.

In [4]:
# Basic callsign prefix to approximate coordinates mapping
# This is a simplified approach - real implementation would use QRZ, HamDB, or grid square data
CALLSIGN_REGIONS = {
    # US Callsign areas (Kx, Nx, Wx, AAx-ALx)
    'K1': {'lat': 42.3601, 'lon': -71.0589, 'name': 'New England'},
    'K2': {'lat': 40.7128, 'lon': -74.0060, 'name': 'New York/New Jersey'},
    'K3': {'lat': 39.9526, 'lon': -75.1652, 'name': 'Mid-Atlantic'},
    'K4': {'lat': 33.7490, 'lon': -84.3880, 'name': 'Southeast'},
    'K5': {'lat': 29.7604, 'lon': -95.3698, 'name': 'South Central'},
    'K6': {'lat': 34.0522, 'lon': -118.2437, 'name': 'California'},
    'K7': {'lat': 47.6062, 'lon': -122.3321, 'name': 'Pacific Northwest'},
    'K8': {'lat': 41.4993, 'lon': -81.6944, 'name': 'Great Lakes'},
    'K9': {'lat': 41.8781, 'lon': -87.6298, 'name': 'Midwest'},
    'K0': {'lat': 39.0997, 'lon': -94.5786, 'name': 'Central Plains'},
    # European prefixes
    'G': {'lat': 51.5074, 'lon': -0.1278, 'name': 'England'},
    'F': {'lat': 48.8566, 'lon': 2.3522, 'name': 'France'},
    'DL': {'lat': 52.5200, 'lon': 13.4050, 'name': 'Germany'},
    'I': {'lat': 41.9028, 'lon': 12.4964, 'name': 'Italy'},
    'EA': {'lat': 40.4168, 'lon': -3.7038, 'name': 'Spain'},
    'ON': {'lat': 50.8503, 'lon': 4.3517, 'name': 'Belgium'},
    'PA': {'lat': 52.3676, 'lon': 4.9041, 'name': 'Netherlands'},
    'OH': {'lat': 60.1699, 'lon': 24.9384, 'name': 'Finland'},
    'SM': {'lat': 59.3293, 'lon': 18.0686, 'name': 'Sweden'},
    'LA': {'lat': 59.9139, 'lon': 10.7522, 'name': 'Norway'},
    'HB': {'lat': 46.9480, 'lon': 7.4474, 'name': 'Switzerland'},
    'LX': {'lat': 49.6116, 'lon': 6.1319, 'name': 'Luxembourg'},
    'GM': {'lat': 55.9533, 'lon': -3.1883, 'name': 'Scotland'},
    'EI': {'lat': 53.3498, 'lon': -6.2603, 'name': 'Ireland'},
    'CT': {'lat': 38.7223, 'lon': -9.1393, 'name': 'Portugal'},
    'CN': {'lat': 33.5731, 'lon': -7.5898, 'name': 'Morocco'},
    # Other regions
    'VE': {'lat': 45.4215, 'lon': -75.6972, 'name': 'Canada'},
    'VA': {'lat': 43.6532, 'lon': -79.3832, 'name': 'Canada'},
    'JA': {'lat': 35.6762, 'lon': 139.6503, 'name': 'Japan'},
    'ZS': {'lat': -25.7461, 'lon': 28.1881, 'name': 'South Africa'},
    'VK': {'lat': -33.8688, 'lon': 151.2093, 'name': 'Australia'},
    'ZL': {'lat': -41.2865, 'lon': 174.7762, 'name': 'New Zealand'},
    'PY': {'lat': -23.5505, 'lon': -46.6333, 'name': 'Brazil'},
    'LU': {'lat': -34.6037, 'lon': -58.3816, 'name': 'Argentina'},
    'CE': {'lat': -33.4489, 'lon': -70.6693, 'name': 'Chile'},
    'XE': {'lat': 19.4326, 'lon': -99.1332, 'name': 'Mexico'},
    'C5': {'lat': 9.0820, 'lon': -79.5199, 'name': 'Gambia'},
    'C6': {'lat': 25.0343, 'lon': -77.3963, 'name': 'Bahamas'},
    '9Y': {'lat': 10.6918, 'lon': -61.2225, 'name': 'Trinidad'},
    'KP4': {'lat': 18.2208, 'lon': -66.5901, 'name': 'Puerto Rico'},
    'J8': {'lat': 17.1274, 'lon': -61.8468, 'name': 'St. Vincent'},
    'T7': {'lat': 13.7942, 'lon': -88.8965, 'name': 'San Marino'},
    'TO': {'lat': 14.6928, 'lon': -17.4467, 'name': 'Martinique'},
    'PZ': {'lat': 5.8520, 'lon': -55.2038, 'name': 'Suriname'},
    '3G': {'lat': -14.9333, 'lon': -145.3833, 'name': 'Easter Island'},
    '9H': {'lat': 35.8989, 'lon': 14.5146, 'name': 'Malta'},
    'EG': {'lat': 40.4168, 'lon': -3.7038, 'name': 'Spain'},
    'IU': {'lat': 41.9028, 'lon': 12.4964, 'name': 'Italy'},
    'IK': {'lat': 45.4642, 'lon': 9.1900, 'name': 'Italy'},
    'IZ': {'lat': 41.9028, 'lon': 12.4964, 'name': 'Italy'},
    'IW': {'lat': 41.9028, 'lon': 12.4964, 'name': 'Italy'},
    'DK': {'lat': 52.5200, 'lon': 13.4050, 'name': 'Germany'},
    'F4': {'lat': 48.8566, 'lon': 2.3522, 'name': 'France'},
    'F5': {'lat': 48.8566, 'lon': 2.3522, 'name': 'France'},
    'F8': {'lat': 48.8566, 'lon': 2.3522, 'name': 'France'},
}

def get_coordinates(callsign):
    """Extract approximate coordinates from callsign prefix."""
    if not callsign:
        return None
    
    # Try progressively shorter prefixes
    for length in [3, 2, 1]:
        prefix = callsign[:length].upper()
        if prefix in CALLSIGN_REGIONS:
            return CALLSIGN_REGIONS[prefix]
    
    # Handle US callsigns (N/W/K/A prefix with digit)
    # Examples: N3NTJ (digit at pos 1), KE4HVR (digit at pos 2)
    first_char = callsign[0].upper()
    if first_char in ['N', 'W', 'K', 'A'] and len(callsign) > 1:
        # Try digit at position 1
        if callsign[1].isdigit():
            key = f'K{callsign[1]}'
            if key in CALLSIGN_REGIONS:
                return CALLSIGN_REGIONS[key]
        # Try digit at position 2 (for KE4, KF4, etc.)
        if len(callsign) > 2 and callsign[2].isdigit():
            key = f'K{callsign[2]}'
            if key in CALLSIGN_REGIONS:
                return CALLSIGN_REGIONS[key]
    
    return None

print("Testing coordinate lookup:")
test_calls = ['K6MKF', 'EA5ZZ', 'IK2CKR', 'KP4YAT', 'CT1BWU', 'CN8VY', 'KE4HVR', 'N3NTJ', 'KF4WE']
for call in test_calls:
    coords = get_coordinates(call)
    if coords:
        print(f"  {call}: {coords['name']} ({coords['lat']:.2f}, {coords['lon']:.2f})")
    else:
        print(f"  {call}: NOT FOUND")

Testing coordinate lookup:
  K6MKF: California (34.05, -118.24)
  EA5ZZ: Spain (40.42, -3.70)
  IK2CKR: Italy (45.46, 9.19)
  KP4YAT: Puerto Rico (18.22, -66.59)
  CT1BWU: Portugal (38.72, -9.14)
  CN8VY: Morocco (33.57, -7.59)
  KE4HVR: Southeast (33.75, -84.39)
  N3NTJ: Mid-Atlantic (39.95, -75.17)
  KF4WE: Southeast (33.75, -84.39)


## Process Spots and Create GreatCircle Data

Convert API spots into a format suitable for pydeck GreatCircleLayer.

In [5]:
# Process spots and create arc data
arc_data = []

for spot in spots:
    # Get coordinates for both stations
    spotter_coords = get_coordinates(spot['spotter_call'])
    dx_coords = get_coordinates(spot['dx_call'])
    
    if spotter_coords and dx_coords:
        arc = {
            'source_lat': spotter_coords['lat'],
            'source_lon': spotter_coords['lon'],
            'target_lat': dx_coords['lat'],
            'target_lon': dx_coords['lon'],
            'spotter': spot['spotter_call'],
            'dx_station': spot['dx_call'],
            'frequency': spot['frequency'],
            'mode': spot.get('mode', 'N/A'),
            'timestamp': spot['timestamp'],
            'comment': spot.get('comment', ''),
        }
        arc_data.append(arc)

print(f"Created {len(arc_data)} arcs from {len(spots)} spots")
print(f"Success rate: {len(arc_data)/len(spots)*100:.1f}%" if spots else "No spots to process")

# Create DataFrame
df = pd.DataFrame(arc_data)

if len(df) > 0:
    print("\nFirst few arcs:")
    print(df[['spotter', 'dx_station', 'frequency', 'mode']].head())
else:
    print("\nNo valid arcs created. Check if coordinates are available for the callsigns.")

Created 244 arcs from 365 spots
Success rate: 66.8%

First few arcs:
  spotter dx_station  frequency  mode
0   N9GXA     XE2ARP  29600.000  None
1   KF4WE       NZ9N  29600.000  None
2   KF4WE     N7GB/M  29600.000  None
3   KF4WE     WA1YKL  29600.000  None
4  WA3LKT       N5PP  29600.000  None


## Create Pydeck Visualization

Create an interactive 3D globe visualization with GreatCircle arcs showing the radio contacts.

In [6]:
if len(df) > 0:
    # Define GreatCircle layer
    layer = pdk.Layer(
        'GreatCircleLayer',
        data=df,
        get_source_position=['source_lon', 'source_lat'],
        get_target_position=['target_lon', 'target_lat'],
        get_source_color=[255, 100, 0, 200],  # Orange for source
        get_target_color=[0, 128, 255, 200],  # Blue for target
        get_width=5,
        pickable=True,
        auto_highlight=True,
    )
    
    # Define view state - center on Atlantic Ocean to see US-EU contacts
    view_state = pdk.ViewState(
        latitude=40,
        longitude=-30,
        zoom=2.5,
        pitch=0,
        bearing=0,
    )
    
    # Create tooltip
    tooltip = {
        "html": "<b>{spotter}</b> → <b>{dx_station}</b><br/>"
                "Frequency: {frequency} kHz<br/>"
                "Mode: {mode}<br/>"
                "Time: {timestamp}<br/>"
                "Comment: {comment}",
        "style": {
            "backgroundColor": "steelblue",
            "color": "white",
            "fontSize": "12px",
            "padding": "10px"
        }
    }
    
    # Create deck with a light map style (no token required)
    deck = pdk.Deck(
        layers=[layer],
        initial_view_state=view_state,
        tooltip=tooltip,
        map_style='light',  # Simple map without requiring Mapbox token
    )
    
    # Try to display inline (may not work in VS Code)
    try:
        deck.show()
        print(f"\n✓ Visualization created with {len(df)} contacts!")
    except:
        print(f"\n✓ Visualization object created with {len(df)} contacts!")
    
    # Save to HTML file for viewing in browser
    html_file = '10m_fm_contacts_map.html'
    deck.to_html(html_file)
    print(f"✓ Saved interactive map to: {html_file}")
    print(f"  Open this file in a web browser to see the visualization")
    print("  Hover over arcs to see contact details.")
else:
    print("No data to visualize. Try adjusting the query parameters or date range.")


✓ Visualization created with 244 contacts!
✓ Saved interactive map to: 10m_fm_contacts_map.html
  Open this file in a web browser to see the visualization
  Hover over arcs to see contact details.


## Summary Statistics

In [7]:
if len(df) > 0:
    print(f"Total Contacts Visualized: {len(df)}")
    print(f"\nUnique DX Stations: {df['dx_station'].nunique()}")
    print(f"Unique Spotters: {df['spotter'].nunique()}")
    print(f"\nModes:")
    print(df['mode'].value_counts())
    print(f"\nTop 5 Most Spotted Stations:")
    print(df['dx_station'].value_counts().head())
    print(f"\nTop 5 Most Active Spotters:")
    print(df['spotter'].value_counts().head())
else:
    print("No data available for statistics.")

Total Contacts Visualized: 244

Unique DX Stations: 103
Unique Spotters: 139

Modes:
mode
USB    18
SSB     3
Name: count, dtype: int64

Top 5 Most Spotted Stations:
dx_station
CT9/UR9IDX    21
WP4CRG        15
9Y4M           9
IW2MJQ         8
N2KHH/VY2      7
Name: count, dtype: int64

Top 5 Most Active Spotters:
spotter
IK3ORE     8
NU1O       8
KF4WE      7
K8WEE      7
OH0M-44    6
Name: count, dtype: int64


## Notes

- **FM Frequency Range**: This notebook queries spots in the 29.6-29.7 MHz range (the 10m FM calling frequencies)
- **Mode Field**: Since mode is not currently stored in the database, we identify FM contacts by frequency range
- **Coordinates**: Uses approximate coordinates based on callsign prefixes for quick visualization
- **Production Enhancement**: For production use, integrate with QRZ.com API, HamDB, or use Maidenhead grid squares for accurate locations
- **GreatCircle Layer**: Shows the great circle path (shortest distance) between two points on the globe
- **Fallback**: If no FM contacts are found, the notebook falls back to showing all 10m contacts
- **Interactivity**: Hover over arcs to see detailed information about each contact